In [1]:
import numpy as np
import pandas as pd
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

In [2]:
first_import = pd.read_csv('./clean_frame.csv',delimiter='\t', low_memory=False)
first_import = first_import.drop(['Unnamed: 0'],axis=1)
print first_import
print first_import.columns

      user_id  gender_yr4  citizen_yr4  workhours_yr4  tradeexp_yr4  \
0        -1.0         1.0         -1.0            5.0           3.0   
1         2.0         1.0          1.0            4.0           2.0   
2         4.0         1.0          1.0            6.0           1.0   
3         5.0        -1.0          1.0            4.0           3.0   
4         7.0         1.0          1.0            4.0           3.0   
5         8.0         1.0          1.0            4.0           1.0   
6         9.0         1.0          1.0            2.0           3.0   
7        10.0         1.0          1.0            1.0           3.0   
8        11.0         1.0          1.0            1.0           2.0   
9        12.0         1.0         -1.0            6.0           2.0   
10       13.0         1.0          1.0            5.0           1.0   
11       14.0         1.0          1.0            5.0           3.0   
12       15.0         1.0         -1.0            4.0           2.0   
13    

In [32]:
g = first_import.columns.to_series().groupby(first_import.dtypes).groups
print g

{dtype('float64'): Index([u'user_id', u'gender_yr4', u'citizen_yr4', u'workhours_yr4',
       u'tradeexp_yr4', u'tradeenjoy_yr4', u'newshrs_yr4', u'raven_score_yr4',
       u'crt_score_yr4', u'crt_1_cor_yr4',
       ...
       u'motivation_2', u'motivation_3', u'motivation_4', u'motivation_5',
       u'motivation_6', u'motivation_7', u'motivation_8',
       u'motivation_intrinsic_score', u'motivation_prosocial_score',
       u'motivation_extrinsic_score'],
      dtype='object', length=168)}


In [33]:
user_id_latent_vars = pd.read_csv('latent_vectors.csv', header=None,delimiter=' ', low_memory=False)

In [34]:
user_id_latent_vars.insert(0, 'user_id', range(len(user_id_latent_vars)))
print user_id_latent_vars

      user_id         0         1         2         3         4
0           0  0.801119  0.282122 -0.069982  0.088061  0.012564
1           1  0.281693  0.140392  0.035321  0.071278  0.116399
2           2 -1.426946 -0.496993 -0.154872 -0.658918 -1.117059
3           3  1.555524 -1.041844 -1.051234 -0.784799  0.045633
4           4 -0.658844  1.023427 -0.920273 -0.071181  0.031747
5           5  0.047146  0.004539  0.168807  0.178217  0.135203
6           6 -0.265250  0.116643 -0.696398  0.972239  0.119027
7           7  0.032499  0.018015  0.555362  1.665917  0.097454
8           8 -0.142926 -0.267900 -0.363851  0.290011  0.123941
9           9 -0.187400  1.656197 -0.506597  0.848599  0.037224
10         10 -0.699145 -0.339964  1.302983  0.381225  0.067343
11         11 -1.342690  1.005448 -0.288503  0.545845  0.048617
12         12 -0.194383 -0.017577  0.327792  0.117514  0.122522
13         13 -0.097314  1.063165 -0.333109 -0.319708  0.089516
14         14  1.702321 -1.458340  0.209

In [35]:
merged_data = pd.merge(user_id_latent_vars,first_import,on='user_id')

In [36]:
print merged_data

      user_id         0         1         2         3         4  gender_yr4  \
0           2 -1.426946 -0.496993 -0.154872 -0.658918 -1.117059         1.0   
1           4 -0.658844  1.023427 -0.920273 -0.071181  0.031747         1.0   
2           5  0.047146  0.004539  0.168807  0.178217  0.135203        -1.0   
3           7  0.032499  0.018015  0.555362  1.665917  0.097454         1.0   
4           8 -0.142926 -0.267900 -0.363851  0.290011  0.123941         1.0   
5           9 -0.187400  1.656197 -0.506597  0.848599  0.037224         1.0   
6          10 -0.699145 -0.339964  1.302983  0.381225  0.067343         1.0   
7          11 -1.342690  1.005448 -0.288503  0.545845  0.048617         1.0   
8          12 -0.194383 -0.017577  0.327792  0.117514  0.122522         1.0   
9          13 -0.097314  1.063165 -0.333109 -0.319708  0.089516         1.0   
10         14  1.702321 -1.458340  0.209531  0.314762  0.053497         1.0   
11         15 -0.075937 -0.086714  0.522809  0.38482

In [37]:
np_merged = merged_data.values

In [38]:
print np_merged.shape

(1711, 173)


In [39]:
def build_model(input_dim, mid_dimension, output_dim):
    model = torch.nn.Sequential()
    # computes w_c^T x + b_c 
    model.add_module("linear1",
                     torch.nn.Linear(input_dim, mid_dimension))
    model.add_module("softmax1", torch.nn.LogSoftmax())
    model.add_module("linear2",
                     torch.nn.Linear(mid_dimension, output_dim))
    # NEEDS TO BE MODIFIED FOR REGRESSION HERE
    # Compute our log softmax term.
    #model.add_module("softmax2", torch.nn.LogSoftmax())
    return model

In [83]:
def build_linear_model(input_dim, output_dim):
    model = torch.nn.Sequential()
    # computes w_c^T x + b_c 
    model.add_module("linear1",
                     torch.nn.Linear(input_dim, output_dim))
    # NEEDS TO BE MODIFIED FOR REGRESSION HERE
    # Compute our log softmax term.
    #model.add_module("softmax2", torch.nn.LogSoftmax())
    return model

In [40]:
def NLLLossForNN(y_pred,target):
    loss = nn.NLLLoss(size_average=True)
    loss = loss.forward(y_pred,target)
    return loss

In [41]:
def train(model, loss, optimizer, x_val, y_val):

    # Resets the gradients to 0
    optimizer.zero_grad()

    # Computes the forward pass
    fx = model.forward(x_val)

    # Computes a loss. Gives a scalar. 
    #output = custom_loss(fx,y_val,lam,model)
    output = loss.forward(fx, y_val)

    # Magically computes the gradients. 
    output.backward()

    # updates the weights
    optimizer.step()
    return output.data[0]

In [42]:
def predict(model, x_val):
    x = Variable(x_val, requires_grad=False)
    output = model.forward(x)
    return output.data.numpy().argmax(axis=1)

def validate_performance(test_model, test_iter, batch_size, text_field):
    batch_size = 100
    n_examples=1000
    num_batches = n_examples // batch_size
    num_correct = 0
    for batch in test_iter:
        x = bag_of_words(batch, text_field)
        y = batch.label - 1
        x = x.type(torch.FloatTensor)
        predY = predict(test_model, x.data)
        num_correct += (predY == y.data.numpy()).sum()
    print num_correct
    return float(num_correct) / n_examples

In [43]:
torch_data_set = Variable(torch.from_numpy(np_merged)).float()
print torch_data_set

Variable containing:
    2.0000    -1.4269    -0.4970  ...      5.0000     5.0000     6.0000
    4.0000    -0.6588     1.0234  ...      5.0000     4.0000     2.0000
    5.0000     0.0471     0.0045  ...      5.0000     5.0000     2.0000
              ...                  ⋱                 ...               
 2049.0000     0.6419    -1.0243  ...     -1.0000    -1.0000    -1.0000
 2050.0000     0.5810    -0.4255  ...     -1.0000    -1.0000    -1.0000
 2051.0000    -0.3281     1.0978  ...     -1.0000    -1.0000    -1.0000
[torch.FloatTensor of size 1711x173]



In [44]:
Y = torch_data_set[:,1:6]
print Y

Variable containing:
-1.4269e+00 -4.9699e-01 -1.5487e-01 -6.5892e-01 -1.1171e+00
-6.5884e-01  1.0234e+00 -9.2027e-01 -7.1181e-02  3.1747e-02
 4.7146e-02  4.5393e-03  1.6881e-01  1.7822e-01  1.3520e-01
                             ⋮                              
 6.4190e-01 -1.0243e+00  1.5391e+00 -3.4980e-01  4.7726e-02
 5.8103e-01 -4.2554e-01  1.3730e+00 -5.1685e-01  5.3839e-02
-3.2809e-01  1.0978e+00 -3.9209e-01 -2.0013e-01  7.4289e-02
[torch.FloatTensor of size 1711x5]



In [45]:
X = torch_data_set[:,6:]
print X

Variable containing:
  1.0000   1.0000   4.0000  ...    5.0000   5.0000   6.0000
  1.0000   1.0000   6.0000  ...    5.0000   4.0000   2.0000
 -1.0000   1.0000   4.0000  ...    5.0000   5.0000   2.0000
           ...               ⋱              ...            
  1.0000  -1.0000  -1.0000  ...   -1.0000  -1.0000  -1.0000
 -1.0000  -1.0000  -1.0000  ...   -1.0000  -1.0000  -1.0000
  1.0000  -1.0000  -1.0000  ...   -1.0000  -1.0000  -1.0000
[torch.FloatTensor of size 1711x167]



In [91]:
n_input_features = 167
#model = build_model(167, 20, 5)
model = build_linear_model(167,5)
X_train = X[:1600,:]
Y_train = Y[:1600,:]
X_test = X[1600:,:]
Y_test = Y[1600:,:]
# Loss here is negative log-likelihood 
loss = torch.nn.MSELoss(size_average=True)

# Optimizer. SGD stochastic gradient. 
#optimizer = optim.SGD(model.parameters(), lr=0.001)
optimizer = optim.Adam(model.parameters(), lr=0.001)

for i in range(3000):
    optimizer.zero_grad()
    cost = 0.
    predictions = model.forward(X_train)
    output = loss(predictions,Y_train)
    if (i% 100 == 0):
        print output
    output.backward()
    optimizer.step()

test_predictions = model.forward(X_test)
test_loss = loss(test_predictions,Y_test)
print "test loss: {}".format(test_loss)
    

Variable containing:
 5.4636
[torch.FloatTensor of size 1]

Variable containing:
 0.6885
[torch.FloatTensor of size 1]

Variable containing:
 0.6238
[torch.FloatTensor of size 1]

Variable containing:
 0.5970
[torch.FloatTensor of size 1]

Variable containing:
 0.5819
[torch.FloatTensor of size 1]

Variable containing:
 0.5722
[torch.FloatTensor of size 1]

Variable containing:
 0.5657
[torch.FloatTensor of size 1]

Variable containing:
 0.5610
[torch.FloatTensor of size 1]

Variable containing:
 0.5576
[torch.FloatTensor of size 1]

Variable containing:
 0.5550
[torch.FloatTensor of size 1]

Variable containing:
 0.5530
[torch.FloatTensor of size 1]

Variable containing:
 0.5514
[torch.FloatTensor of size 1]

Variable containing:
 0.5501
[torch.FloatTensor of size 1]

Variable containing:
 0.5490
[torch.FloatTensor of size 1]

Variable containing:
 0.5480
[torch.FloatTensor of size 1]

Variable containing:
 0.5472
[torch.FloatTensor of size 1]

Variable containing:
 0.5465
[torch.Floa

In [86]:
print torch.cat((test_predictions[:,None],Y_test[:,None]),1)

Variable containing:
( 0 ,.,.) = 
 -3.9312e-01  1.7826e-01 -3.6497e-01 -9.8802e-02 -5.7866e-02
 -1.1173e-01 -1.3616e+00  1.1527e+00 -1.1476e+00  6.2809e-02

( 1 ,.,.) = 
  6.1557e-05  3.7740e-02  5.0136e-02  2.8594e-02  2.1963e-02
 -5.0659e-01 -1.6591e+00 -1.5809e+00 -1.3423e-01  6.1843e-02

( 2 ,.,.) = 
 -1.8808e-01 -2.8942e-01 -1.4693e-02 -2.2434e-01  1.0631e-01
 -1.2672e-01 -3.8011e-02 -3.1110e-01 -6.1887e-01  1.0551e-01
... 

(108,.,.) = 
  3.7123e-01  1.2083e-01  7.1512e-02  4.0618e-02  8.7529e-02
  6.4190e-01 -1.0243e+00  1.5391e+00 -3.4980e-01  4.7726e-02

(109,.,.) = 
  2.8142e-02 -1.0107e-01 -3.0308e-01  2.6784e-01  1.4673e-01
  5.8103e-01 -4.2554e-01  1.3730e+00 -5.1685e-01  5.3839e-02

(110,.,.) = 
  3.5099e-01 -3.1905e-01 -2.4546e-02 -6.4883e-02  1.0461e-01
 -3.2809e-01  1.0978e+00 -3.9209e-01 -2.0013e-01  7.4289e-02
[torch.FloatTensor of size 111x2x5]



In [87]:
print test_predictions

Variable containing:
-0.3931  0.1783 -0.3650 -0.0988 -0.0579
 0.0001  0.0377  0.0501  0.0286  0.0220
-0.1881 -0.2894 -0.0147 -0.2243  0.1063
 0.0569 -0.0628  0.1712 -0.0623  0.1143
-0.0067  0.7831  0.1283 -0.3355 -0.3628
 0.1567 -0.0832  0.1425  0.0966  0.0802
 0.2188 -0.0540  0.0943  0.0903  0.0130
 0.3081  0.0010  0.2661  0.2001 -0.0546
-0.1475  0.4655 -0.6251 -0.4862  0.1983
-0.2098 -0.3423 -0.1717 -0.1268 -0.0838
 0.4677 -0.3481  0.0600  0.5179  0.2539
 0.0849 -0.0260  0.1169  0.1332  0.0789
 0.0455  0.3744  0.0511  0.2807  0.0338
 0.1214 -0.0148 -0.0075 -0.1194 -0.0614
 0.0965  0.0615 -0.0456  0.1660 -0.0078
 0.1811  0.0060  0.2601 -0.2290  0.0620
 0.0099 -0.0692 -0.0671 -0.0019 -0.0239
 0.0186 -0.0732 -0.2143  0.0943  0.0187
-0.0169 -0.0680  0.0837  0.1497  0.0226
-0.0913 -0.1824 -0.3890 -0.1498  0.1456
 0.2798  0.2737  0.3177 -0.1940 -0.0619
-0.0320  0.2268 -0.2042  0.0529 -0.0224
 0.0075  0.2648 -0.3638  0.2336 -0.0042
-0.0774  0.1185  0.1036  0.0846  0.0050
-0.0162  0.0890 -0.

In [88]:
print Y_test

Variable containing:
-0.1117 -1.3616  1.1527 -1.1476  0.0628
-0.5066 -1.6591 -1.5809 -0.1342  0.0618
-0.1267 -0.0380 -0.3111 -0.6189  0.1055
 1.1695  1.3561 -0.4970 -0.2358  0.0594
 1.3159  0.1674 -0.7314  0.5151  0.0982
 0.4503  0.2029 -0.4257 -0.2145  0.0043
-0.1562 -1.7057  1.1194 -0.4583  0.0351
-0.4063 -0.0839  0.5053 -1.3062  0.0888
 0.4985  0.3682  0.5580 -0.0514  0.0639
-0.2034 -0.0460 -0.0629  0.0582  0.0661
-1.0441 -0.7427 -0.1084 -1.2437  0.0891
-0.0367  0.3842 -0.9453  0.3831  0.1079
 0.4663  0.5803  0.3176 -0.2842 -1.4774
-0.0982  0.1031  0.1757  0.1882  0.1268
-1.4037  0.8361  0.6983 -0.0054  0.0711
-0.6215  0.8151 -1.9055 -0.6149  0.0770
 1.1939  1.1505  0.0246  2.2948  0.0272
 0.1221 -0.1918  0.2806 -0.3169  0.1286
-0.1751 -1.0088  0.0676  0.8882  0.0443
 0.2358  0.4527  1.4851  1.4688  0.0236
 0.5357 -0.2584  0.7606 -0.3229  0.0738
 0.1621 -0.8559  0.8153  0.2077  0.0988
 0.0486  0.0310 -0.0010  0.0455  0.0642
-0.0055  0.3864  1.4007 -0.3664  0.0698
 0.2561 -0.2796 -0.

In [89]:
easy_print = torch.cat((test_predictions[:,None],Y_test[:,None]),1)

In [90]:
print easy_print[3:8]

Variable containing:
(0 ,.,.) = 
  0.0569 -0.0628  0.1712 -0.0623  0.1143
  1.1695  1.3561 -0.4970 -0.2358  0.0594

(1 ,.,.) = 
 -0.0067  0.7831  0.1283 -0.3355 -0.3628
  1.3159  0.1674 -0.7314  0.5151  0.0982

(2 ,.,.) = 
  0.1567 -0.0832  0.1425  0.0966  0.0802
  0.4503  0.2029 -0.4257 -0.2145  0.0043

(3 ,.,.) = 
  0.2188 -0.0540  0.0943  0.0903  0.0130
 -0.1562 -1.7057  1.1194 -0.4583  0.0351

(4 ,.,.) = 
  0.3081  0.0010  0.2661  0.2001 -0.0546
 -0.4063 -0.0839  0.5053 -1.3062  0.0888
[torch.FloatTensor of size 5x2x5]

